In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import netCDF4
from scipy import stats
from scipy.stats import norm

***
# Functions for processing data

In [ ]:
########################################################################
#                                                                      #
########################################################################
def read_csv(filename):
    """ read in the csv files we previously made of UTH anomaly time series.
    inputs: filename -> full path to file containing our data

    ouputs: results -> python dictionary containing our anomaly time series.
    """
    results = {}
    data = np.loadtxt(filename, skiprows=1, delimiter=",",dtype='<f4')
    header = np.genfromtxt(filename, delimiter=',', dtype=str, max_rows=1)
    for ii, varname in enumerate(header):
        results[varname] = data[:,ii]
    return results


In [ ]:
########################################################################
#                                                                      #
########################################################################
def simple_moving_average(yvals, width):
    """ compute the moving average of a time series with a user defined sliding window.
    inputs: yvals -> time series on regular time steps
            width -> width of sliding window (n time steps)
    output: sommothed time series
    """
    return np.convolve(yvals, np.ones(width), 'same')/width

In [ ]:
########################################################################
#                                                                      #
########################################################################
def find_enso_periods(smoothed_ts,enso_idx,dates,nmonths,phase='Elnino'):

    cnt, index, periods, max_anom = 0,[],[],[]
    for ii, val in enumerate(enso_idx):
        if val != 0:
            cnt+=1
            index.append(ii)
        else:
            if cnt == 0:
                pass
            else:
                if cnt >=nmonths:
                    periods.append(dates[index])
                    if phase == 'Elnino':
                        max_anom.append(max(smoothed_ts[index]))
                    else:
                        max_anom.append(min(smoothed_ts[index]))
                cnt = 0
                index = []
    ####################################################################
    years = []
    for dd in periods:
        st = f"{int(dd[0])}"
        fn = f"{int(dd[-1])}"
        years.append(f"{st}/{fn}")

    return periods, max_anom, years

In [ ]:
##################################################################################################
def calculate_probDensFunc(yvals, ymin, ymax,nbins=100):
    """ calculate a PDF from the array yvals using scipy.stats norm module
    inputs: yvals -> 1d array of values to be used in PDF calculation
            ymin  -> minimum value for range of yvals PDF is calculated
            ymax  -> maximum value for range of yvals PDF is calculated
            nbins -> number of bins for which PDF is calculated between ymin  and ymax
            
    outputs: xvals -> array of values over which the PDF was calculated (defined by ymin, ymax,nbins)
             PDF   -> array containing PDF values
    """
    # define xvals
    xvals = np.linspace(ymin,ymax,nbins)

    # calculate mean and standard deviation of yvals
    mu = np.mean(yvals)
    std = np.std(yvals)

    # define an empty array to hold PDF values. Here we use the size method to tell the code how many elements 
    # are in this new array and fill each entry with a default value NaN (Not a Number)
    PDF = np.full(xvals.size, np.nan)

    # loop over each value in x and calculate the corresponding PDF value. Because xvals is an array merans in 
    # Python it is iterable (i.e. we can loop over the contents) and by wrapping it in the enumerate function
    # we also get the index of the value (e.g. the firts value in xvals could be -2, therefore x=-2 and ii=0).
    for ii, x in enumerate(xvals):
        PDF[ii] = norm.pdf(x, loc=mu, scale=std)

    # return the results
    return PDF, xvals

In [ ]:
def plot_enso_phase_pdfs(xvals, el_pre_pdf, el_aft_pdf, la_pre_pdf, la_aft_pdf, vname):
    """ plot pdf of either uth or lst"""
    plt.plot(xvals, el_pre_pdf, ":", lw=2,color='#1F3649',alpha=0.7,label='El Nino (1979-2000)')
    plt.plot(xvals, la_pre_pdf, ":",lw=2,color="#2E75B6",alpha=0.7,label='La Nina (1979-2020)')
    
    plt.plot(xvals, el_aft_pdf,lw=2,color='#1F3649',label='El Nino (2001-2024)')
    plt.plot(xvals, la_aft_pdf,lw=2,color="#2E75B6",label='La Nina (2001-2024)')
    if vname == "lst":
        plt.xlim(-2,3)
        plt.ylim(0,1.6)
        plt.xlabel(r"$\Delta$LST [K]",fontsize=12)
    elif vname == 'uth':
        plt.xlim(-3,2)
        plt.ylim(0,1.5)
        plt.xlabel(r"$\Delta$UTH [%]",fontsize=12)
    else:
        pass
    plt.ylabel("PDF",fontsize=12)
    plt.legend(loc=1,fontsize=8)

In [ ]:
def plot_uth_vs_Ts(ts_anom, uth_anom, region1, region2):
    #coeffs = estimate_coef(ts_anom, uth_anom)
    slope, intercept, r, p, se = stats.linregress(ts_anom, uth_anom)
    # calculate the pearson coeeficients
    #R = stats.pearsonr(ts_anom, uth_anom)
    # convert pearson coefficient to a string 
    Rs = f"{r:0.2f}"
    if p < 0.01:
        Rs+="*"

    vmin =-1.5
    vmax = 2.5
    # plot scatter
    plt.plot(ts_anom, uth_anom, 'o',color="#2E75B6",mec='#1F3649')
    # plot fit
    xg = np.linspace(vmin,vmax,50)
    yg = intercept+slope*xg
    plt.plot(xg, yg,'--',color="#C55A11",lw=2,label=f"{slope:0.2f}"+r"$\pm$"+f"{se:0.2f} %/K")
    plt.ylabel(f"{region1} "+r"$\Delta$UTH (%)")
    plt.xlabel(f'{region2} '+r"$\Delta$T$_{s}$ (K)")
    plt.title(f'R={Rs}')
    plt.ylim(-2.5,1.5)
    plt.xlim(vmin,vmax)
    plt.legend()
    plt.grid(True)

In [ ]:
##################################################################################################
def read_gridded_data(filename, report=True):
    """ function to read regular gridded data files and return a Python dictionary.
    inputs: filename -> name of file to be read
            report   -> boolean flag, if true then a table listing the file contents is printed to screen

    outputs: data    -> a python dictionary containing the file contents
    """
    # open the file and map the contents to a netCDF object. Use a test to capture
    # any issues with the data file
    try:
        nc = netCDF4.Dataset(filename,"r")
    except IOError:
        raise
    # if report is set to true then prind global attributes
    if report == True:
        print(f"Global Attributes For File: {filename}") # when witing strings, starting them with an 'f' allows you to insert other variables using {}.
        print("----------------------------------------------------------------------------------------------------")
        print(nc)
    # Define a dictionary to hold the contents
    data = {} 
    # loop over the file contents and write
    if report == True:
        # what variables are in this file?
        # printing Aligned Header 
        print("-------------------------------------------------------------------")        
        print(f"{'Variable Name' : <14} |{'Long Name':<32} |{'tdim, ydim, xdim':>17}") 
        print("-------------------------------------------------------------------")
    else:
        pass # we add a pass so we can close this if statement with an else

    for varname in nc.variables.keys():
        if report == True:
            print(f"{varname:<14} |{nc[varname].long_name:<32} |{', '.join([str(d) for d in nc[varname].shape]):>17}")
        else:
            pass
        # write the variable to the dictionary
        data[varname] = nc[varname][:] #  the [:] at the end indexes all the data
    
    # finally close the file
    nc.close()

    # return the filled data dictionary
    return data

In [ ]:
########################################################################
#                                                                      #
########################################################################
def convert_lon(lons,reverse=0):
    if reverse == 0:
        return np.fmod((lons + 360), 360.0)
    else:
        return np.fmod((lons + 180), 360.0) - 180

########################################################################
#                                                                      #
########################################################################
def create_enso_timeseries(tskin, lats, lons, lsm):

    # 1. create 2D lons and lats
    lons2d, lats2d = np.meshgrid(lons,lats)

    # calculate the weights
    wgts = np.cos(np.radians(lats2d))
    
    ####################################################################
    # NOW WE CALCULATE THE SST TIME SERIES FOR DIFFERENT ENSO REGIONS
    #
    # 2. define ENSO regions
    # nino regions [min_x, max_x, min_y, max_y]
    # for longitude grid that runs from 0-360
    regions={'Nino_1+2':[convert_lon(-90),
                         convert_lon(-80),
                         -10,0],
             'Nino_3':[convert_lon(-150),
                       convert_lon(-90),
                       -5, 5],
             'Nino_3.4':[convert_lon(-170),
                        convert_lon(-120),
                        -5, 5],
             'Nino_4':[convert_lon(160),
                   convert_lon(-150),
                   -5, 5]}
    # 3. calculate gridded tskin anomolies
    tdim, ydim, xdim = tskin.shape
    enso_ts = {}
   
    
    # loop over each time step and andcalculated weighted mean 
    for tt, grid in enumerate(tskin):
        mdx = tt % 12
        for reg in regions:
            if tt == 0:
                enso_ts[reg] = np.full(tdim, np.nan)
            else:
                pass
            bbox = regions[reg]
            find = np.where((lons2d >= bbox[0])&\
                            (lons2d <= bbox[1])&\
                            (lats2d >= bbox[2])&\
                            (lats2d <= bbox[3])&\
                            (lsm == 0))
            
            wgts = np.cos(np.radians(lats2d[find]))
            enso_ts[reg][tt] = np.nansum(grid[find]*wgts)/np.nansum(wgts)
            


    # calculate weighted mean sst anomoly time series
    sst_anomolies = {}
    for reg in regions:
        sst_anomolies[reg] = np.full(tdim, np.nan)
        for mth in range(12):
            sst_anomolies[reg][mth::12] = enso_ts[reg][mth::12] - np.nanmean(enso_ts[reg][mth::12])
            
    return sst_anomolies

In [ ]:
########################################################################
#                                                                      #
########################################################################
def mad(x):
    """ calculate the medain absolute deviation """
    return np.median(abs(x-np.median(x)))

########################################################################
#                                                                      #
########################################################################
def robust_zscore(x):
    """ calculate the robust z scores for a set of data."""
    return 0.6745*(x - np.median(x))/mad(x)

***
# define data files to be used
- This is all the data we are using in this example notebook. Tou will notcice that some of the filepaths have got a ```../``` infront of them . This tells the computer to go up one level in the file directory before decending into the correct directory

In [ ]:
# example 1: Anomaly time series 
anom_ts_filename = "data/uth_ts_anomaly_timeseries.csv"

# example 2 + 3: calculate ENSO ssts from ERA5 data
lsm_filename = "../PA260X_CourseWorkMaterials/data/pa260x_ecmwf_Land_sea_mask_2.5_x_2.5_1950_2020.nc"
Ts_filelist= ["../PA260X_CourseWorkMaterials/data/pa260x_ecmwf_Skin_temperature_2.5_x_2.5_1950_1989.nc",
              "../PA260X_CourseWorkMaterials/data/pa260x_ecmwf_Skin_temperature_2.5_x_2.5_1990_2020.nc"]

# eample 4: filetring data with robust z scores
tcwv_filename = "data/reyk_suominet_gps_tcwv_2008_2020.csv"

***
# Set out the workflow
# Example 1: plot PDF and scatter plot example using previous CW rsults
- In this example we will recreate a plot similar to one we saw in the last lecture. I have already exported all the data you need into a csv file.
- Lets start by reading that in

In [ ]:
#1. read in the time series
anom_data = read_csv(anom_ts_filename)

- next list the file contents

In [ ]:
# list the file contents
sorted(anom_data.keys())

- the next task is to identify the La Nina and El Nino relevant data.
- I have included a function to this <span color="blue">**find_enso_periods**</span> to do this. Don't worry if isn't clear, its not something we have covered in workshops or lectures. Later we will resue it to calculate an ENSO phase index that you can use for your course work. 

In [ ]:
# 2. seperate the data out the enso phases
#######################################
# detect ENSO phases, we are going to use the ONI (Oceanic Nino Index) 
smth_win = 3
thresh=[-0.5,0.5]
nmonths=5

# smooth the time series with running average of 2 months
smoothed_ts = simple_moving_average(anom_data['sea_surface_temperature_anomaly_enso_3.4'], smth_win)

# detect months where the sst anomoly values exceed the
# threshold values. This produces 2 array of 1s and 0s
lanina = (smoothed_ts < thresh[0]).astype(int)
elnino = (smoothed_ts > thresh[1]).astype(int)

el_periods, el_max_anom, el_years = find_enso_periods(smoothed_ts,
                                                      elnino,
                                                      anom_data['frac_year'], 
                                                      nmonths,
                                                      phase='Elnino')

la_periods, la_max_anom, la_years = find_enso_periods(smoothed_ts,
                                                      lanina,
                                                      anom_data['frac_year'], 
                                                      nmonths,
                                                      phase='Lanina')

In [ ]:
# 3. seperate out data into el nino and la nina phases
la_results_lst = []
la_results_uth=[]
la_yrs = []

# first we loop throughthe identified La Nina events
for phase in la_periods:
    for fy in phase:
        # match the dates to extract the index in the anomaly tiumeseries arrays
        idx = abs(anom_data['frac_year']-fy).argmin()
        la_results_lst.append(anom_data['smoothed_land_surface_temperature_anomaly_tropics'][idx])
        la_results_uth.append(anom_data['smoothed_uth_anomaly_tropics_land'][idx])
        la_yrs.append(int(fy))
        

el_results_lst = []
el_results_uth= []
el_yrs = []
for phase in el_periods:
    for fy in phase:
        # match the dates to extract the index in the anomaly tiumeseries arrays
        idx = abs(anom_data['frac_year']-fy).argmin()
        el_results_lst.append(anom_data['smoothed_land_surface_temperature_anomaly_tropics'][idx])
        el_results_uth.append(anom_data['smoothed_uth_anomaly_tropics_land'][idx])
        el_yrs.append(int(fy))


- then we can split the identified La Nina and El Nino data into rpe and after 2001 using a ```np.where``` statement

In [ ]:
# 3. create index values for La Nina and El Nino phases, pre and post 2001
##########################################################################
la_pre2000 = np.where(np.array(la_yrs) < 2001)
la_aft2000 = np.where(np.array(la_yrs) >= 2001)

el_pre2000 = np.where(np.array(el_yrs) < 2001)
el_aft2000 = np.where(np.array(el_yrs) >= 2001)

- calculate PDFs reusing code from the first workshop

In [ ]:
# 4. Calculate PDFs
#########################################################################################################
# LST
pre_la_lst_pdf, xvals_lst = calculate_probDensFunc(np.array(la_results_lst)[la_pre2000], -2, 5, nbins=80)
aft_la_lst_pdf, xvals_lst = calculate_probDensFunc(np.array(la_results_lst)[la_aft2000], -2, 5, nbins=80)
pre_el_lst_pdf, xvals_lst = calculate_probDensFunc(np.array(el_results_lst)[el_pre2000], -2, 5, nbins=80)
aft_el_lst_pdf, xvals_lst = calculate_probDensFunc(np.array(el_results_lst)[el_aft2000], -2, 5, nbins=80)

# UTH
pre_la_uth_pdf, xvals_uth = calculate_probDensFunc(np.array(la_results_uth)[la_pre2000], -5, 3, nbins=80)
aft_la_uth_pdf, xvals_uth = calculate_probDensFunc(np.array(la_results_uth)[la_aft2000], -5, 3, nbins=80)
pre_el_uth_pdf, xvals_uth = calculate_probDensFunc(np.array(el_results_uth)[el_pre2000], -5, 3, nbins=80)
aft_el_uth_pdf, xvals_uth = calculate_probDensFunc(np.array(el_results_uth)[el_aft2000], -5, 3, nbins=80)

- finally plot the results
- You will notice we now have functions to plot the data and our code below is tidier and easy to follow


In [ ]:
# 5. plot the results
############################################################################################################

plt.figure(figsize=(13,4),dpi=300)

plt.subplot(131)
plot_enso_phase_pdfs(xvals_lst, pre_el_lst_pdf, aft_el_lst_pdf, pre_la_lst_pdf, aft_la_lst_pdf, 'lst')
plt.title(r"LST Anomalies over the Tropics")


plt.subplot(132)
plot_uth_vs_Ts(anom_data['smoothed_land_surface_temperature_anomaly_tropics'], 
               anom_data['smoothed_uth_anomaly_tropics_land'],'(Tropical Land)', '(Tropical Land)')

plt.subplot(133)
plot_enso_phase_pdfs(xvals_uth, pre_el_uth_pdf, aft_el_uth_pdf, pre_la_uth_pdf, aft_la_uth_pdf, 'uth')
plt.title(r"UTH Anomalies Over the Tropics")

plt.tight_layout()

#plt.savefig("uth_results_summary.png")

***
# Example 2: caluclate ENSO 3.4 SST anomalies from coursework data


- In this example we are going to calculate the SST anomolies for the 4 ENSO regions usingthe coursework surface temeprature dataset. Lets start by reading in the Land Sea Mask (LSM) and the surface skin temperature datasets. Like with all NetCDF files we use our <span style="color:blue">**read_gridded_data()**</span>. Lets start with the LSM file

In [ ]:
lsm_data = read_gridded_data(lsm_filename)

- with the report flag set to <span style="color:green">**True**</span> we get the information on the file contents. We can see that there is just the LSM, longitude, and latitude information. For the skin temperature we have 2 files which we will need to combine. Lets read in the first file in the list and see what is inside: 

In [ ]:
ts_data = read_gridded_data(Ts_filelist[0])

- now we can see all the variables we can write the code to join up the time series. We have 2 variables that need to be joined and they are ```skt``` (with is 3D) and ```time``` (which is 1D).
- Below I have edit some code from the second computer workshop where we stacked AVHRR reflectance data.
- for the skin temeprature ```skt``` we use  ```np.vstack``` to join the data in the time dimension, and for ```time``` we just use ```np.append``` to append the arrays together  

In [ ]:
# let now loop over each file, read the contents, and stack the results
for ii, fname in enumerate(sorted(Ts_filelist)):
    # because we can iterate overlists in Python, we canuse the enumerate() 
    # method to give us the positional index at the same time (e.g. 0,1,2,3..).
    # We have also use the sorted() method to force the list into chronological
    # order - which happens because the filenames have been formatted with the 
    # date in YYYYMMDD at the end of the filename.  
    
    if ii == 0:
        # for the first file we assign it to the dictionary avhrr_data with Ch1 and Ch2 variables 
        # dimensions of (1,900,1800)
        ts_data = read_gridded_data(fname, report=False)
    else:
        # for every other file we stack the contents of Ch1 and Ch2 on top such that the dimensions
        # avhrr_data[Ch1] and avhrr_data[Ch2] are (2,900,1800) when the second file is read in, 
        # (3,900,1800) when the third is read in and so on until the final dimensions are (10,900,1800)
        
        tmp = read_gridded_data(fname, report=False) # hold output in a temporary dictionarry
        ts_data['skt'] = np.vstack([ts_data['skt'], tmp['skt']])
        ts_data['time'] = np.append(ts_data['time'], tmp['time'])

        # delete the temporary dictionary
        del tmp

- now we have the data we can calculate the SST anomolies for the ENSO regions (we haven't covered this code  previously)

In [ ]:
enso_ssts = create_enso_timeseries(ts_data['skt'], ts_data['latitude'], ts_data['longitude'], lsm_data['lsm'])

- Lets plot the data so we can examine it

In [ ]:
# simple four panel plot of ENSO region SST anomalies
plt.figure(figsize=(7,4),dpi=200)
for pp, enso_reg in enumerate(enso_ssts.keys()):
    plt.subplot(2,2,pp+1)
    x = ts_data['time']
    y = np.full(x.size,0)
    plt.plot(x,y,'--r')
    plt.plot(x, enso_ssts[enso_reg])
    plt.title(enso_reg)
    plt.ylim(-3,5)
    plt.xlim(1950,2021)
    plt.ylabel(r"$\Delta$SST [K]")
plt.tight_layout()

- Next we can determine which monthly time steps are in either the El Nino, neutral, or La Nina phase. For this we apply the Oceanic Nino Index (ONI) technique. Anomalies are calculated for specific regions for a specific reference period so we can then apply a 2 month moving average filter to smooth the data. Threshold is then set to $\pm$0.5 K.
- We haven't covered this before so don't worry if it does not seem clear 

In [ ]:
# 2. seperate the data out the enso phases
#######################################
# detect ENSO phases, we are going to use the ONI (Oceanic Nino Index) 
smth_win = 3
thresh=[-0.5,0.5]
nmonths=5

# smooth the time series with running average of 2 months
smoothed_ts = simple_moving_average(enso_ssts['Nino_3.4'], smth_win)

# detect months where the sst anomoly values exceed the
# threshold values. This produces 2 array of 1s and 0s
lanina = (smoothed_ts < thresh[0]).astype(int)
elnino = (smoothed_ts > thresh[1]).astype(int)

el_periods, el_max_anom, el_years = find_enso_periods(smoothed_ts,
                                                      elnino,
                                                      ts_data['time'], 
                                                      nmonths,
                                                      phase='Elnino')

la_periods, la_max_anom, la_years = find_enso_periods(smoothed_ts,
                                                      lanina,
                                                      ts_data['time'], 
                                                      nmonths,
                                                      phase='Lanina')

- Lets make an index array that can be used later to filter other timeseries calcuylated from the coursework datasets

In [ ]:
enso_phase_index = np.full(ts_data['time'].size,0)
enso_phase_index[lanina == 1] = -1
enso_phase_index[elnino == 1] = 1

- lets make a plot to illustrate the enso pahse index array we have just made

In [ ]:
plt.figure(figsize=(7,2),dpi=200)
plt.fill_between(x, enso_phase_index, where=enso_phase_index >=0, facecolor="#C55A11",interpolate=True)
plt.fill_between(x, enso_phase_index, where=enso_phase_index < 0, facecolor="#2E75B6",interpolate=True)
plt.plot(x, enso_phase_index,color='#1F3649')
plt.title('Enso Phase Index')
plt.ylim(-1.1,1.1)
plt.xlim(1950,2021)
plt.ylabel(r"Index")


- The code below is a slightly modified version of the plotting code in the cell above and we visualise our SST anomolies from the Nino 3.4 region

In [ ]:
plt.figure(figsize=(7,2),dpi=200)
plt.fill_between(x, smoothed_ts, where=smoothed_ts>=0, facecolor="r",interpolate=True)
plt.fill_between(x, smoothed_ts, where=smoothed_ts < 0, facecolor="b",interpolate=True)
plt.plot(x,smoothed_ts,color='#1F3649')
plt.title('Nino 3.4')
plt.ylim(-3, 3)
plt.xlim(1950,2021)
plt.ylabel(r"$\Delta$SST")


## Now We can save this for use another time! We reuse a piece of code to write csv files with a small update

In [ ]:
##################################################################################################
# This our code to save the outputs
# we open a file called uth_anomoly_timeseries.csv and assign it to a file object we have named fobj
# we can the loop over each time step and write the results to file
with open("enso_3.4_sst_anomaly.csv", "w") as fobj:
    # first we write a header, the \n tells the computer to start a new line
    fobj.write("Frac_Year, SST_anomaly_(NINO_3.4), enso_phase_index\n")
    # here we use zip in order to interate over all arrays at the same time
    for yr, v1, v2 in zip(ts_data['time'], enso_ssts['Nino_3.4'], enso_phase_index):
        # we create a f string and write it to file
        fobj.write(f"{yr:0.3f}, {v1:0.2f}, {v2:0.2f}\n")

***
# Example 3: Correlation to climate indices
- In this example we will calculate the correlation to climate indicies and plot a map
- We will look at the correlation between our smoothed Nino 3.4 SST anomolies and surface skin temperature
- we will use the ```stats.pearsonr``` from scipy for this task. You could do something similar to calculate trends using ```stats.linregress```

In [ ]:
# define our dims
tdim, ydim, xdim = ts_data['skt'].shape

# set up an array to hold our results
corr_to_enso = np.full((ydim,xdim),np.nan)
sig_corr = np.full((ydim,xdim),np.nan)

# now we loop over the y and x dimensions and calculate the correlation between the skin temperature (skt) ans the ENSO SST anomolies
for j in range(ydim):
    for i in range(xdim):
        # calculate correlation for grid cell (j,i)
        res = stats.pearsonr(ts_data['skt'][:,j,i], smoothed_ts)
        # write correlation result to array
        corr_to_enso[j,i] = res.statistic
        # if significant mark the poitionin the sig_corr array with a 1
        if res.pvalue < 0.05:
            sig_corr[j,i] = 1

In [ ]:
plt.figure(figsize=(12,5))
plt.pcolormesh(ts_data['longitude'], ts_data['latitude'], corr_to_enso, vmin=-1, vmax=1,cmap=plt.get_cmap('PiYG', 16))
plt.colorbar()
plt.contour(ts_data['longitude'], ts_data['latitude'],lsm_data['lsm'],levels=[0,1],cmap='Greys_r')
plt.title("Correlation between ERA5 skin temperature and Nino3.4")

- Now we can make a nice plot of the results lets add the statistcial significance to the plot with stippling.
- Essentially we will plot a small dot if the data is statistciall significant. The results of this are held in the ```sig_corr``` array, which eith has a 1 or NaN entry for each grid cell.
- We can use the ```np.ma.masked_invalid``` function to mask the NaN values in the ```sig_corr``` array. If we then create 2D arrays of longitude and latitude we can apply the same mask to them.
- Finally, we use the ```.compressed()``` method onour newly masked 2D arrays of long itude and latitude to turn them back to 1D and just polt the points.

In [ ]:
# create 2D arrays of longigude and latitude
x2d, y2d = np.meshgrid(ts_data['longitude'], ts_data['latitude'])
# mask the sig_corr array
sig_corr = np.ma.masked_invalid(sig_corr)
# apply the mask to the 2D lons and lats, then compress
x2d = np.ma.masked_array(x2d,mask=sig_corr.mask).compressed()
y2d = np.ma.masked_array(y2d,mask=sig_corr.mask).compressed()

# plot the same figure from before
plt.figure(figsize=(12,5))
plt.pcolormesh(ts_data['longitude'], ts_data['latitude'], corr_to_enso, vmin=-1, vmax=1,cmap=plt.get_cmap('PiYG', 16))
plt.colorbar()
plt.contour(ts_data['longitude'], ts_data['latitude'],lsm_data['lsm'],levels=[0,1],cmap='Greys_r')
plt.title("Correlation between ERA5 skin temperature and Nino3.4")
# now add the stippling to indicate areas of statisitical significance in the correlations
plt.plot(x2d,y2d,'.',ms=1,color='k')

- We can easily apply the same process to skin temperature trends.
- First we calculate the anomalies relative to the 1950-1990 average 

In [ ]:
# set up an array to hold our results
skt_trend = np.full((ydim,xdim),np.nan)
sig_corr = np.full((ydim,xdim),np.nan)

# calculate Tskin anomalies relative to 1950-1980
anom = np.full((tdim, ydim,xdim),np.nan)
mkr = np.where((ts_data['time'] >= 1950)&(ts_data['time'] < 1990))
for mth in range(12):
    anom[mth::12,:,:] = ts_data['skt'][mth::12,:,:] - np.mean(ts_data['skt'][mkr[0],:,:][mth::12,:,:],axis=0)

- we then just repeat the same code structure and preplace the pearson correlation function from scipy with linregress
- the other difference is we scale the slope from per month to per decade

In [ ]:
# now we loop over the y and x dimensions and calculate the linear fit between the skin temperature and time
for j in range(ydim):
    for i in range(xdim):
        # calculate correlation for grid cell (j,i)
        res = stats.linregress(ts_data['time'],anom[:,j,i])
        # write slope result to array, scale fromper month to per decade
        skt_trend[j,i] = res.slope*120
        # if significant mark the poitionin the sig_corr array with a 1
        if res.pvalue < 0.005:
            sig_corr[j,i] = 1

In [ ]:
# create 2D arrays of longigude and latitude
x2d, y2d = np.meshgrid(ts_data['longitude'], ts_data['latitude'])
# mask the sig_corr array
sig_corr = np.ma.masked_invalid(sig_corr)
# apply the mask to the 2D lons and lats, then compress
x2d = np.ma.masked_array(x2d,mask=sig_corr.mask).compressed()
y2d = np.ma.masked_array(y2d,mask=sig_corr.mask).compressed()

# plot the same figure from before
plt.figure(figsize=(12,5))
plt.pcolormesh(ts_data['longitude'], ts_data['latitude'], skt_trend, vmin=-6, vmax=6,cmap=plt.get_cmap('coolwarm', 12))
plt.colorbar(extend='both')
plt.contour(ts_data['longitude'], ts_data['latitude'],lsm_data['lsm'],levels=[0,1],cmap='Greys_r')
plt.title("ERA5 skin Temperature Trend (K/decade)")
# now add the stippling to indicate areas of statisitical significance of the trends
plt.plot(x2d,y2d,'.',ms=1,color='k')

### What can we infer?

***
# Example 4: robust z scores
![](img/suominet_info.png)

 - Lets start by reading in the data. The file is in csv format so we can reuse our <span style="color:blue">**read_csv()**</span> funtion:

In [ ]:
#1. read in the time series
gps_data = read_csv(tcwv_filename)

### Next we can create a simple time series to look at the data

In [ ]:
# produce a simple time series plot of the data
plt.figure(figsize=(12,3),dpi=200)
plt.plot(gps_data['frac_year'], gps_data['tcwv'],'o',ms=1)
plt.ylabel(r"TCWV (kg/m$^{2}$)")
plt.xlim(2008,2020)

- In this example we obsevre a shift in the data between late 2016 and mid-2018, which appears to be biased high releative to the rest of the data. The question is how can we tes that this is true and filter the data if required?

## The robsut z score

- A **robust Z-score** is a modified version of the standard Z-score that helps detect outliers while being less sensitive to extreme values and skewed distributions. Instead of using the mean and standard deviation, robust Z-scores are calculated using the **median** and **median absolute deviation (MAD)**:

# $z_{\text{robust}} = \frac{0.6745(x - median)}{MAD}$

- here MAD is a robust measure of how spread out a set of data is about the median:

### $MAD = medain(|x - median(x)|)$ 

- The factor 0.6745 ensures consistency with the standard deviation for normally distributed data. Values of $( |Z_{\text{robust}}| > 3 )$ typically indicate potential outliers. Robust Z-scores are particularly useful when dealing with non-normal data or datasets containing extreme values, as they provide a more reliable way to identify outliers than traditional Z-scores.



- To represent these as functions we would use the following python code:

```python
    def mad(x):
        """ calculate the medain absolute deviation """
        return np.median(abs(x-np.median(x)))
    
    def robust_zscore(x):
        """ calculate the robust z scores for a set of data."""
        return 0.6745*(x - np.median(x))/mad(x)

```
- These have already been included in the functions above, so we can have a go at filtering the data. First we pass our TCWV data to the  <span style="color:blue">**robust _zscore()**</span> function:

In [ ]:
# calculate the z scores
z = robust_zscore( gps_data['tcwv'])

- Then we can make a simple plot to look at our results and see if we have a cluster of data outside $\pm$3 threshold:

In [ ]:
# plot a histogram of absolute values 
h = plt.hist(abs(z), range=(0,7),bins=100) # here we use the python function abs() which converts all values to absolute o positive values
ymax = np.ceil(h[0].max()/10)*10
plt.plot([3,3],[0,ymax],'--r')
plt.xlim(0,7)
plt.ylim(0,ymax)
plt.ylabel("Number of points")
plt.xlabel("Z Score ")

- we observe a population of the data to the righ hand side of the red dashed line on the plot. Therefore, there is a good chance this is our outlier data. To find outr we write a simple ```np.where``` statement to identify the z scores within the $\pm$3 range.

In [ ]:
find = np.where(abs(z) <3) # here we use the python function abs() which converts all values to absolute o positive values

- Finally lets plot the original data in a light grey and then over plot the filtered TCWV data over the top and see how well it worked:

In [ ]:
# produce a simple time series plot of the data
plt.figure(figsize=(12,3),dpi=200)
plt.plot(gps_data['frac_year'], gps_data['tcwv'],'o', color='lightgrey',ms=1,alpha=0.3, label='unfiltered')
plt.plot(gps_data['frac_year'][find], gps_data['tcwv'][find],'o',ms=1,label='filtered')
plt.ylabel(r"TCWV (kg/m$^{2}$)")
plt.xlim(2008,2020)
plt.legend(loc=2,markerscale=5)

### How well has this worked?

***
# Bonus examples: Climate indicies
- You may want to use a different climate index for you analysis (e.g. qbo for stratospheric temperature)
- In the data folder there is a file called ```climate_indicies.csv``` with ENSO 3.4, NAO, QBO, and PDO
- Below is an example where i have read them in and made simple time series plots 


In [ ]:
#1. read in the time series
clim_inds = read_csv("data/climate_indicies.csv")

In [ ]:
plt.figure(figsize=(7,8),dpi=200)
for ii, ci in enumerate(['qbo', 'pdo', 'nao', 'enso3.4']):
    plt.subplot(4,1,ii+1)
    plt.fill_between(clim_inds['frac_year'], clim_inds[ci], where=clim_inds[ci]>= 0, facecolor="r",interpolate=True)
    plt.fill_between(clim_inds['frac_year'], clim_inds[ci], where=clim_inds[ci] < 0, facecolor="b",interpolate=True)
    plt.plot(clim_inds['frac_year'],clim_inds[ci],color='#1F3649')
    plt.title(ci.upper()) # we can capitalise the string with the .upper() method
    if ci == 'qbo':
        plt.ylim(-35, 35)
    else:
        plt.ylim(-3.5,3.5)
    plt.xlim(1950,2021)
    plt.ylabel(r"Index")
plt.tight_layout()